In [1]:
import operator
import os
import collections
import pickle
import autocomplete2
from collections import Counter
from bottle import route, run, debug
from autocomplete2 import predict
from autocomplete2 import models
from autocomplete2 import helpers
from autocomplete2 import split_predict
from autocomplete2 import predict_currword
from autocomplete2 import predict_second_word_given_first
import re 
import numpy as np 
import time
import csv 
from random import sample

In [2]:
class TrieNode(): 
    def __init__(self): 
          
        # Initialising one node for trie 
        self.children = {} 
        self.last = False
  
class Trie(): 
    def __init__(self): 
          
        # Initialising the trie structure. 
        self.root = TrieNode() 
        self.word_list = [] 
  
    def formTrie(self, keys): 
          
        # Forms a trie structure with the given set of strings 
        # if it does not exists already else it merges the key 
        # into it by extending the structure as required 
        for key in keys: 
            self.insert(key) # inserting one key to the trie. 
  
    def insert(self, key): 
          
        # Inserts a key into trie if it does not exist already. 
        # And if the key is a prefix of the trie node, just  
        # marks it as leaf node. 
        node = self.root 
        for a in list(key): 
        ### If the letter of the prefix is not found in the child nodes, add them to the trie ##
            if not node.children.get(a): 
                node.children[a] = TrieNode() 
            node = node.children[a] 
        node.last = True
  
    def search(self, key): 
          
        # Searches the given key in trie for a full match 
        # and returns True on success else returns False. 
        node = self.root 
        found = True
  
        for a in list(key): 
            if not node.children.get(a):  ### If the key is not found in the child nodes,  breaks the loop. 
                found = False
                break
  
            node = node.children[a]  ### Creates a new node with the letter not found in the child nodes.###
  
        return node and node.last and found 
  
    def suggestionsRec(self, node, word): 
          
        # Method to recursively traverse the trie 
        # and return a whole word.  
        if node.last: 
            self.word_list.append(word) 
        ## Add characters to the word in word list as found in the child nodes ###
        for a,n in node.children.items(): 
            self.suggestionsRec(n, word + a) ##### Adds the letters in the child nodes of the current node (node corresponding to the 
                                             #####last letter of the prefix) to the prefix and then appends it into the 
                                             ##### list of suggestions ####
    
    def printAutoSuggestions(self, key,WORDS,top_n): 
          
        # Returns all the words in the trie whose common 
        # prefix is the given key thus listing out all  
        # the suggestions for autocomplete. 
        self.word_list = []
        node = self.root 
        not_found = False
        temp_word = '' 
  
        for a in list(key): 
           ### If the key is not found in the children, break the loop ###
            if not node.children.get(a): 
                not_found = True
                break
           ### Add the keys to the temp_word string to form the prefix to be searched ###
            temp_word += a 
            ### Call the node corresponding to the current letter of the prefix if the character exists in the child nodes ###
            node = node.children[a] 
        
        if not_found: 
            return 0
        elif node.last and not node.children: ### If the current node itself is the root node (no characters have been added to the trie 
            return -1                         ##structure)##
        print(self.word_list)
        freq_dict  = Counter(WORDS)
        dict = {}
        for s in self.word_list:
            dict[s] = freq_dict[s]
        sorted_tup = sorted(dict.items(), key=operator.itemgetter(1),reverse = True)
        for tup in sorted_tup[:top_n]:
            print(tup[0])
        return 1

In [14]:
corpus = open("everything_combined.txt").readlines()

In [4]:
mean_query_insert_list = []
for length in range(0,len(corpus),40000):
    query_list = sample(corpus,length)
    keys = query_list
    k = 0
    elapsed_list_1 = []
    while k<1:
        t = Trie() 
        start = time.time()
        t.formTrie(query_list) 
        end = time.time()
        elapsed_time = end-start
        elapsed_list_1.append(elapsed_time)
        k += 1
    mean_query_insert_list.append(np.mean(elapsed_list_1))
print(mean_query_insert_list)

[4.291534423828125e-07, 3.65037145614624, 7.026572442054748, 10.799125862121581, 14.71916675567627, 18.846915531158448, 23.57284059524536, 32.79412527084351, 31.390541243553162, 37.48743934631348, 41.97919049263]


In [13]:
t = Trie() 
start = time.time()
t.formTrie(corpus) 
end = time.time()
print("Elapsed time=",end-start)   
      
       

Elapsed time= 6.715363025665283


In [10]:
print(np.asarray(range(0,len(corpus),40000)))

[     0  40000  80000 120000 160000 200000 240000 280000 320000 360000
 400000]


In [7]:
data = np.asarray(mean_query_insert_list)
np.save('mean_query_insert_list_everything.npy', data)

In [17]:
keys = corpus
key = "ev" # key for autocomplete suggestions. 
status = ["Not found", "Found"] 
query_elapsed_list1 = []
k= 0 
while k<10:
    start = time.time()
    comp = t.printAutoSuggestions(key,keys,top_n = 10) 
    end = time.time()
    elapsed_time = end-start
    query_elapsed_list1.append(elapsed_time)
    k += 1
print(query_elapsed_list1)
print(comp)
if comp == -1: 
    print("No other strings found with this prefix\n") 
elif comp == 0: 
    print("No string found with this prefix\n")

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[0.011595010757446289, 0.010305166244506836, 0.01139211654663086, 0.0112152099609375, 0.011381149291992188, 0.013096094131469727, 0.013466119766235352, 0.014119148254394531, 0.012277841567993164, 0.013453960418701172]
1


In [ ]:
data = np.asarray(query_elapsed_list1)
np.save('query_elapsed_list1.npy', data)

In [23]:
print(list(range(0,320610,10000)))

[0, 10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000, 110000, 120000, 130000, 140000, 150000, 160000, 170000, 180000, 190000, 200000, 210000, 220000, 230000, 240000, 250000, 260000, 270000, 280000, 290000, 300000, 310000, 320000]
